# Models

## Setup

In [ ]:
import os
from pathlib import Path

from scripts.renv_utils import activate_renv
from var_es_toolbox.data import load_data

activate_renv()
from rpy2.robjects import pandas2ri, globalenv, r
%load_ext rpy2.ipython

In [ ]:
project_dir = Path(os.getcwd()).resolve().parent
# Asset
asset = "DBc1"
globalenv["asset"] = str(asset)

# Retrieve data
data_dir = project_dir / "data"
data_cleaned_name = "refinitiv_data_merged.csv"
date_format = "ISO8601"
data_cleaned = load_data(data_dir / data_cleaned_name, date_format=date_format).dropna()
data_cleaned = data_cleaned.dropna()

# Split data into multiple dataframes
columns = data_cleaned.columns
price_columns = [col for col in columns if not col.endswith("_return") and "supply" not in col.lower() and "demand" not in col.lower()]
return_columns = [col for col in columns if col.endswith("_return")]
supply_demand_columns = [col for col in columns if "supply" in col.lower() or "demand" in col.lower()]

price_df = data_cleaned[price_columns]
return_df = data_cleaned[return_columns]
return_df.columns = return_df.columns.str.replace("_return", "", regex=False)
fund_df = data_cleaned[supply_demand_columns]

futures_returns = data_cleaned[asset].dropna()

# Retrieve r model
models_dir = project_dir / "src" / "var_es_toolbox" / "models"
backtest_dir = project_dir / "src" / "var_es_toolbox" / "backtesting"
hs_path = models_dir / "hs.R"
garch_path = models_dir / "garch.R"
gas_path = models_dir / "gas.R"
hybrid_evt_path = models_dir / "hybrid_evt.R"
caviar_path = models_dir / "caviar" / "caviar.R"
backtest_path = backtest_dir / "backtesting.R"

globalenv['hs_path'] = str(hs_path)
globalenv['garch_path'] = str(garch_path)
globalenv['gas_path'] = str(gas_path)
globalenv['hybrid_evt_path'] = str(hybrid_evt_path)
globalenv['caviar_path'] = str(caviar_path)
globalenv['backtest_path'] = str(backtest_path)

c = 0.05
t = 0.95
p = 1
q = 1
m = 250
n = 250
refit = 50

asset_df = return_df[[asset]]
asset_df.columns = ["Return"]

globalenv['asset_df'] = pandas2ri.py2rpy(asset_df.reset_index())
globalenv['price_df'] = pandas2ri.py2rpy(price_df.reset_index())
globalenv['return_df'] = pandas2ri.py2rpy(return_df.reset_index())
globalenv['c'] = c
globalenv['p'] = p
globalenv['q'] = q
globalenv['m'] = m
globalenv['n'] = n
globalenv['refit'] = refit

In [ ]:
%%R
library(xts)
library(rugarch)
library(psych)
library(ggplot2)   
library(gridExtra)          
library(tseries)          

dates <- tail(asset_df$Date, n)
returns <- xts(tail(asset_df$Return, n), order.by = dates)

# Function to plot all returns
plot_all_returns <- function(data) {
  # Assume the relevant columns are identified (excluding "Date")
  return_cols <- setdiff(names(data), "Date")
  
  # Calculate global y-axis limits
  y_limits <- range(data[return_cols], na.rm = TRUE)
  
  # Create a list to store individual plots
  plot_list <- lapply(return_cols, function(col) {
    ggplot(data, aes_string(x = "Date", y = col)) +
      geom_line(color = "black") +
      labs(title = col, x = "", y = "") +
      theme_minimal() +
      scale_y_continuous(limits = y_limits) # Apply consistent y-axis limits
  })
  
  # Combine all plots into a grid
  do.call(grid.arrange, c(plot_list, ncol = 2)) # Adjust ncol to control layout
}

# Function to create QQ plots for all columns in a dataframe
plot_all_qq <- function(data) {
  # Exclude 'Date' column
  data_cols <- setdiff(names(data), "Date")
  
  # Calculate global y-axis limits for QQ plots
  qq_limits <- quantile(unlist(data[data_cols]), probs = c(0.01, 0.99), na.rm = TRUE)
  
  # Create a list to store individual QQ plots
  plot_list <- lapply(data_cols, function(col) {
    ggplot(data, aes_string(sample = col)) +
      stat_qq() +
      stat_qq_line() +
      labs(title = col, x = "", y = "") +
      theme_minimal() +
      scale_y_continuous(limits = qq_limits) # Apply consistent y-axis limits
  })
  
  # Combine all QQ plots into a grid
  do.call(grid.arrange, c(plot_list, ncol = 2)) # Adjust ncol to control layout
}

# Function to plot all prices
plot_all_timeseries <- function(data) {
  timeseries_cols <- setdiff(names(data), "Date")
  
  # Define a custom matte color palette
  custom_colors <- c("#597BAF", "#E6A157", "#7F9C66", "#C7615D", 
                     "#B39EB5", "#A07C72", "#D69AB3", "#A6A6A6")
  
  # Map colors to column names
  color_mapping <- custom_colors[seq_along(timeseries_cols)]
  names(color_mapping) <- timeseries_cols
  
  # Create the base plot
  p <- ggplot(data, aes(x = Date)) +
    labs(x = "Date", y = "Value") +
    theme_minimal()
  
  # Add each time series as a separate line
  for (col in timeseries_cols) {
    p <- p + geom_line(aes_string(y = col, color = shQuote(col)))
  }
  
  # Finalize the plot with consistent y-axis limits
  p + scale_color_manual(values = color_mapping, name = "Series")
}

plot_all_timeseries <- function(data) {
  timeseries_cols <- setdiff(names(data), "Date")
  
  # Define a custom matte color palette
  custom_colors <- c("#597BAF", "#E6A157", "#7F9C66", "#C7615D", 
                     "#B39EB5", "#A07C72", "#D69AB3", "#A6A6A6")
  
  # Map colors to column names
  color_mapping <- custom_colors[seq_along(timeseries_cols)]
  names(color_mapping) <- timeseries_cols
  
  # Create the base plot
  p <- ggplot(data, aes(x = Date)) +
    labs(x = "Date", y = "Value") +
    theme_minimal() +
    theme(legend.position = "none")  # Remove default legend
  
  # Add each time series as a separate line
  for (col in timeseries_cols) {
    p <- p + geom_line(aes_string(y = col, color = shQuote(col)))
  }
  
  # Prepare legend positions and labels
  legend_df <- data.frame(
    x = rep(max(data$Date) - (max(data$Date) - min(data$Date)) * 0.1, length(timeseries_cols)),
    y = seq(
      from = max(unlist(data[timeseries_cols], use.names = FALSE)) * 0.9,
      to = max(unlist(data[timeseries_cols], use.names = FALSE)) * 0.7,
      length.out = length(timeseries_cols)
    ),
    label = names(color_mapping),
    color = color_mapping
  )
  
  # Add custom legend inside the plot
  p <- p +
    geom_text(data = legend_df, aes(x = x, y = y, label = label, color = label), hjust = 0, size = 3) +
    geom_point(data = legend_df, aes(x = x - (max(data$Date) - min(data$Date)) * 0.01, y = y, color = label), size = 3)
  
  # Finalize the plot with custom colors
  p + scale_color_manual(values = color_mapping, name = "Series")
}

# Function to plot base vs. peak prices in subplots
plot_base_vs_peak <- function(data, base_cols, peak_cols) {
  if (length(base_cols) != length(peak_cols)) {
    stop("Base and peak column lists must have the same length.")
  }
  
  # Calculate global y-axis limits
  all_cols <- c(base_cols, peak_cols)
  y_limits <- range(data[all_cols], na.rm = TRUE)
  
  # Create a list to store individual plots
  plot_list <- lapply(seq_along(base_cols), function(i) {
    base_col <- base_cols[i]
    peak_col <- peak_cols[i]
    
    ggplot(data, aes(x = Date)) +
      geom_line(aes_string(y = base_col), color = "#597BAF", linetype = "solid") +
      geom_line(aes_string(y = peak_col), color = "#E6A157", linetype = "dashed") +
      labs(title = paste(base_col, "vs", peak_col),
           x = "",
           y = "") +
      theme_minimal() +
      scale_y_continuous(limits = y_limits) # Apply consistent y-axis limits
  })
  
  # Combine all plots into a grid
  do.call(grid.arrange, c(plot_list, ncol = 2)) # Adjust ncol to control layout
}

# Define the function for plotting returns, VaR, and ES
plot_var_es <- function(dates, returns, var, es) {
  # Create a data frame for plotting
  plot_data <- data.frame(
    Date = dates,
    Return = returns,
    VaR = var,
    ES = es
  )
  
  # Generate the plot using ggplot2 with labels
  ggplot(plot_data, aes(x = Date)) +
    geom_line(aes(y = Return, color = "Return"), linewidth = 0.7, alpha = 0.8) +    # Actual returns
    geom_line(aes(y = VaR, color = "VaR"), linetype = "dashed", linewidth = 0.7) + # VaR
    geom_line(aes(y = ES, color = "ES"), linetype = "dotted", linewidth = 0.7) +   # ES
    ggtitle("Actual Returns vs. VaR and ES") +
    xlab("Date") +
    ylab("Returns / VaR / ES") +
    theme_minimal() +
    theme(
      plot.title = element_text(hjust = 0.5),
      legend.position = "top"
    ) +
    scale_color_manual(values = c("Return" = "blue", "VaR" = "red", "ES" = "darkorange")) +
    scale_y_continuous(labels = scales::percent) +
    labs(color = "Legend")
}

## Descriptive Statistics

In [ ]:
%%R

desc_price <- psych::describe(price_df[setdiff(names(price_df), "Date")])
print("Descriptive Statistics for prices")
print(desc_price)

desc_return <- psych::describe(return_df[setdiff(names(return_df), "Date")])

# Compute the JB test for each column in return_df
jb_tests <- apply(return_df[setdiff(names(price_df), "Date")], 2, function(column) {
  test <- jarque.bera.test(na.omit(column)) # Perform JB test
  return(test$p.value) # Return the test statistic
})

# Add JB test statistics as a new column to the descriptive stats
desc_return <- cbind(desc_return, JB_Test = jb_tests)

print("Descriptive Statistics for returns")
print(desc_return)

In [ ]:
%%R

plot_all_returns(return_df)

In [ ]:
%%R

price_P_col <- grep("P", names(price_df), value = TRUE)
plot_all_timeseries(price_df[ , c("Date", price_P_col)])

In [ ]:
%%R


price_B_col <- grep("B", names(price_df), value = TRUE)
plot_all_timeseries(price_df[ , c("Date", price_B_col)])

In [ ]:
%%R

plot_base_vs_peak(price_df, price_B_col, price_P_col)

In [ ]:
%%R

plot_base_vs_peak(return_df, price_B_col, price_P_col)

In [ ]:
%%R

plot_all_qq(return_df)

## VaR and ES forecasting

In [ ]:
%%R
source(backtest_path)
source(hs_path)
source(garch_path)
source(hybrid_evt_path)
source(gas_path)
library(caviar)

df <- asset_df

# Define example models
models <- list(
  "HS" = function(df, c, n, m) forecast_u_HS(df, c = c, n = n, m = m),
  "FHS_EWMA" = function(df, c, n, m) forecast_u_FHS_EWMA(df, c = c, n = n, m = m),
  "FHS_sGARCH_norm" = function(df, c, n, m) forecast_u_FHS_GARCH(df, c = c, n = n, m = m, model = "sGARCH", dist = "norm"),
  "FHS_sGARCH_std" = function(df, c, n, m) forecast_u_FHS_GARCH(df, c = c, n = n, m = m, r = 50, model = "sGARCH", dist = "std"),
  "FHS_sGARCH_sstd" = function(df, c, n, m) forecast_u_FHS_GARCH(df, c = c, n = n, m = m, r = 50, model = "sGARCH", dist = "sstd"),
  "FHS_eGARCH_norm" = function(df, c, n, m) forecast_u_FHS_GARCH(df, c = c, n = n, m = m, r = 50, model = "eGARCH", dist = "norm"),
  "FHS_eGARCH_std" = function(df, c, n, m) forecast_u_FHS_GARCH(df, c = c, n = n, m = m, r = 50, model = "eGARCH", dist = "std"),
  "FHS_eGARCH_sstd" = function(df, c, n, m) forecast_u_FHS_GARCH(df, c = c, n = n, m = m, r = 50, model = "eGARCH", dist = "sstd"),
  "FHS_gjrGARCH_norm" = function(df, c, n, m) forecast_u_FHS_GARCH(df, c = c, n = n, m = m, r = 50, model = "gjrGARCH", dist = "norm"),
  "FHS_gjrGARCH_std" = function(df, c, n, m) forecast_u_FHS_GARCH(df, c = c, n = n, m = m, r = 50, model = "gjrGARCH", dist = "std"),
  "FHS_gjrGARCH_sstd" = function(df, c, n, m) forecast_u_FHS_GARCH(df, c = c, n = n, m = m, r = 50, model = "gjrGARCH", dist = "sstd"),
  "sGARCH_norm" = function(df, c, n, m) forecast_u_GARCH(df, c = c, n = n, m = m, r = 50, model = "sGARCH", dist = "norm"),
  "sGARCH_std" = function(df, c, n, m) forecast_u_GARCH(df, c = c, n = n, m = m, r = 50, model = "sGARCH", dist = "std"),
  "sGARCH_sstd" = function(df, c, n, m) forecast_u_GARCH(df, c = c, n = n, m = m, r = 50, model = "sGARCH", dist = "sstd"),
  "eGARCH_norm" = function(df, c, n, m) forecast_u_GARCH(df, c = c, n = n, m = m, r = 50, model = "eGARCH", dist = "norm"),
  "eGARCH_std" = function(df, c, n, m) forecast_u_GARCH(df, c = c, n = n, m = m, r = 50, model = "eGARCH", dist = "std"),
  "eGARCH_sstd" = function(df, c, n, m) forecast_u_GARCH(df, c = c, n = n, m = m, r = 50, model = "eGARCH", dist = "sstd"),
  "gjrGARCH_norm" = function(df, c, n, m) forecast_u_GARCH(df, c = c, n = n, m = m, r = 50, model = "gjrGARCH", dist = "norm"),
  "gjrGARCH_std" = function(df, c, n, m) forecast_u_GARCH(df, c = c, n = n, m = m, r = 50, model = "gjrGARCH", dist = "std"),
  "gjrGARCH_sstd" = function(df, c, n, m) forecast_u_GARCH(df, c = c, n = n, m = m, r = 50, model = "gjrGARCH", dist = "sstd"),
  "EVT_sGARCH_norm" = function(df, c, n, m) forecast_u_EVT_GARCH(df, c = c, n = n, m = m, t = 0.95, r = 50, model = "sGARCH", dist = "norm"),
  "EVT_sGARCH_std" = function(df, c, n, m) forecast_u_EVT_GARCH(df, c = c, n = n, m = m, t = 0.95, r = 50, model = "sGARCH", dist = "std"),
  "EVT_sGARCH_sstd" = function(df, c, n, m) forecast_u_EVT_GARCH(df, c = c, n = n, m = m, t = 0.95, r = 50, model = "sGARCH", dist = "sstd"),
  "EVT_eGARCH_norm" = function(df, c, n, m) forecast_u_EVT_GARCH(df, c = c, n = n, m = m, t = 0.95, r = 50, model = "eGARCH", dist = "norm"),
  "EVT_eGARCH_std" = function(df, c, n, m) forecast_u_EVT_GARCH(df, c = c, n = n, m = m, t = 0.95, r = 50, model = "eGARCH", dist = "std"),
  "EVT_eGARCH_sstd" = function(df, c, n, m) forecast_u_EVT_GARCH(df, c = c, n = n, m = m, t = 0.95, r = 50, model = "eGARCH", dist = "sstd"),
  "EVT_gjrGARCH_norm" = function(df, c, n, m) forecast_u_EVT_GARCH(df, c = c, n = n, m = m, t = 0.95, r = 50, model = "gjrGARCH", dist = "norm"),
  "EVT_gjrGARCH_std" = function(df, c, n, m) forecast_u_EVT_GARCH(df, c = c, n = n, m = m, t = 0.95, r = 50, model = "gjrGARCH", dist = "std"),
  "EVT_gjrGARCH_sstd" = function(df, c, n, m) forecast_u_EVT_GARCH(df, c = c, n = n, m = m, t = 0.95, r = 50, model = "gjrGARCH", dist = "sstd"),
  "GAS_norm" = function(df,c, n, m) forecast_u_GAS(df, c = c, n = n, m = m, r = 50, dist = "norm"),
  "GAS_std" = function(df,c, n, m) forecast_u_GAS(df, c = c, n = n, m = m, r = 50, dist = "std"),
  "GAS_sstd" = function(df,c, n, m) forecast_u_GAS(df, c = c, n = n, m = m, r = 50, dist = "sstd"),
  "CAViaR_ADAPTIVE_AR" = function(df, c, n, m) forecast_u_CAViaR(df, c = c, n = n, m = m, r = 50, var_model = "ADAPTIVE", es_model = "AR"),
  "CAViaR_SAV_AR" = function(df, c, n, m) forecast_u_CAViaR(df, c = c, n = n, m = m, r = 50, var_model = "SAV", es_model = "AR"),
  "CAViaR_AS_AR" = function(df, c, n, m) forecast_u_CAViaR(df, c = c, n = n, m = m, r = 50, var_model = "AS", es_model = "AR"),
  "CAViaR_indirectGARCH_AR" = function(df, c, n, m) forecast_u_CAViaR(df, c = c, n = n, m = m, r = 50, var_model = "indirectGARCH", es_model = "AR"),
  "CAViaR_ADAPTIVE_MULT" = function(df, c, n, m) forecast_u_CAViaR(df, c = c, n = n, m = m, r = 50, var_model = "ADAPTIVE", es_model = "MULT"),
  "CAViaR_SAV_MULT" = function(df, c, n, m) forecast_u_CAViaR(df, c = c, n = n, m = m, r = 50, var_model = "SAV", es_model = "MULT"),
  "CAViaR_AS_MULT" = function(df, c, n, m) forecast_u_CAViaR(df, c = c, n = n, m = m, r = 50, var_model = "AS", es_model = "MULT"),
  "CAViaR_indirectGARCH_MULT" = function(df, c, n, m) forecast_u_CAViaR(df, c = c, n = n, m = m, r = 50, var_model = "indirectGARCH", es_model = "MULT")
)

# Compute losses and backtests
results <- compute_losses_and_backtests(df, models, c, n, m)

# Access results
losses <- results$losses
backtests <- results$backtests
predictions <- results$predictions

c_percent <- as.integer(c * 100)

# Save losses to CSV
write.csv(losses, paste0(asset, "_", c_percent, "_losses.csv"), row.names = TRUE)

# Save backtests to CSV
write.csv(backtests, paste0(asset, "_", ac_percent, "_backtests.csv"), row.names = TRUE)

# Save predictions to CSV
write.csv(predictions, paste0(asset, "_", c_percent, "_predictions.csv"), row.names = TRUE)

# Perform MCS Test and print results
alpha <- 0.05
losses <- losses[, colSums(is.na(losses)) < nrow(losses)]
losses[is.na(losses)] <- 1e6
mcs_results <- rugarch::mcsTest(losses, alpha = alpha, nboot = 500, boot = "stationary")
print(colnames(losses))
print(mcs_results)

### Historical Simulation

In [ ]:
%%R
source(hs_path)

result <- forecast_u_HS(asset_df, c, n, m)

var <- -result$VaR
VaRplot(c, returns, var)

In [ ]:
%%R

es <- -result$ES
print(plot_var_es(dates, returns, var, es))

In [ ]:
%%R
source(backtest_path)

vol <- result$VOL
run_backtests(returns, var, es, c, prefix = "HS")

### Filtered Historical Simulation

In [ ]:
%%R

result <- forecast_u_FHS_GARCH(asset_df, c, n, m, model = "sGARCH", dist = "sstd")

var <- -result$VaR
VaRplot(c, returns, var)

In [ ]:
%%R

es <- -result$ES
print(plot_var_es(dates, returns, var, es))

In [ ]:
%%R
source(backtest_path)

vol <- result$VOL
run_backtests(returns, var, es, c, VOL = vol, prefix = "FHS")

### GARCH

In [ ]:
%%R
source(garch_path)

result <- forecast_u_GARCH(asset_df, c, n, m, r = 50, model = "gjrGARCH", dist = "norm", solver = "hybrid", solver.control = list(trace = 0))

var <- -result$VaR
VaRplot(c, returns, var)

In [ ]:
%%R

es <- -result$ES
print(plot_var_es(dates, returns, var, es))

In [ ]:
%%R
source(backtest_path)

vol <- result$VOL
run_backtests(returns, var, es, c, VOL = vol, prefix = "sGARCH")

### Generalized Autoregressive Score

In [ ]:
%%R
source(gas_path)

result <- forecast_u_GAS(asset_df, c, n, m, r = 1, dist = "std")

var <- -result$VaR
VaRplot(c, returns, var)

In [ ]:
%%R

es <- -result$ES
print(plot_var_es(dates, returns, var, es))

In [ ]:
%%R
source(backtest_path)

vol <- result$VOL
print(run_backtests(returns, var, es, c, prefix = "GAS"))

### EVT Hybrid GARCH

In [ ]:
%%R
source(hybrid_evt_path)

nn <- 300

result <- forecast_u_EVT_GARCH(asset_df, 0.05, 300, m, r = 50, t = 0.9, model = "gjrGARCH", dist = "norm", solver = "hybrid", solver.control = list(trace = 0))

print(result)

returns_s <- tail(returns, 300)

var <- -result$VaR
print(var)
VaRplot(c, returns_s, var)

In [ ]:
%%R
print(warnings())

es <- -result$ES
print(plot_var_es(dates, returns, var, es))

In [ ]:
%%R
source(backtest_path)

vol <- result$VOL
run_backtests(returns, var, es, c, VOL = vol, prefix = "EVT_GARCH")

### CAViaR

In [ ]:
%%R
library(caviar)
# source(caviar_path)

result <- forecast_u_CAViaR(asset_asset_df, c, n, m, r = 10, var_model = "indirectGARCH", es_model="AR", itermax = 250)

var <- -result$VaR
VaRplot(c, returns, var)

In [ ]:
%%R

es <- -result$ES
print(plot_var_es(dates, returns, var, es))
print(var - es)

In [ ]:
%%R
source(backtest_path)

vol <- result$VOL
run_backtests(returns, var, es, c, prefix = "CAVIAR")